In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [59]:
EPOCHS = 20

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    # "..\\dataset\\train",
    "..\\dataset_new\\train",
    shuffle = True,
    batch_size = 32,
)

Found 2784 files belonging to 3 classes.


In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
plt.figure(figsize=(10,10))
for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        ax= plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(class_names[label_batch[i]])

In [64]:
def get_dataset_partitions_tf(ds, train_split=0.9, val_split=0.1, shuffle = True, shuffle_size=10000):
    ds_size = len(ds)

    if shuffle: 
        ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split*ds_size) 
    val_size = int(val_split*ds_size)

    train_ds = ds.take(train_size) 
    val_ds = ds.skip(train_size).take(val_size) 

    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
                # "..\\dataset\\test",
                "..\\dataset_new\\test",
                shuffle = True,
                batch_size = 32,
            )

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
len(train_ds)

In [ ]:
len(val_ds)

In [ ]:
len(test_ds)

In [69]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [70]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(256,256),
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255),
])

In [3]:
tf.__version__

'2.13.1'

In [72]:
data_augementation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [73]:
input_shape = (256, 256, 3)
num_classes = 3

# model = tf.keras.models.Sequential([
#     resize_and_rescale,
#     data_augementation,
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2,2)),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(num_classes, activation='softmax'),
# ])

# VGG16 model
# vgg_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=input_shape)

# ResNet model
resnet_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

for layer in resnet_model.layers:
    layer.trainable=False

model = tf.keras.models.Sequential([
    # resize_and_rescale,
    # data_augementation,
    resnet_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
                        #   kernel_regularizer=tf.keras.regularizers.l2(0.01)), # new

    # tf.keras.layers.Dropout(0.3), # new
    tf.keras.layers.Dense(num_classes, activation='softmax'),

])

model.build(input_shape=(None,256,256,3))

In [ ]:
model.summary()

In [75]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy'],
)

In [263]:
# early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True) # new

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=32,
    verbose=1,
    validation_data=val_ds,
    # callbacks=[early_stopping_callback] # new
)

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
scores

In [ ]:
history

In [ ]:
history.params

In [81]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8)) 
plt.subplot(1, 2, 1) 
plt.plot(range (EPOCHS), acc, label='Training Accuracy') 
plt.plot(range (EPOCHS), val_acc, label='Validation Accuracy') 
plt.legend(loc='lower right') 
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2) 
plt.plot(range (EPOCHS), loss, label='Training Loss') 
plt.plot(range (EPOCHS), val_loss, label='Validation Loss') 
plt.legend(loc='upper right') 
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1): 
    first_image = images_batch[0].numpy().astype('uint8') 
    first_label = labels_batch[0].numpy() 
    print("first image to predict")
    plt.imshow(first_image) 
    print("actual label:", class_names[first_label]) 
    batch_prediction = model.predict(images_batch) 
    print("predicted label: ", class_names[np.argmax(batch_prediction[0])])

In [86]:
def predict (model, img): 
    img_array = tf.keras.preprocessing.image.img_to_array(img) 
    img_array = tf.expand_dims(img_array, 0) # Create a batch 

    predictions = model.predict(img_array) 
    predicted_class = class_names[np.argmax(predictions[0])] 
    confidence = round(100 * (np.max(predictions[0])), 2) 
    
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15)) 
for images, labels in test_ds.take(1): 
    for i in range(9): 
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8")) 
        predicted_class, confidence = predict(model, images[i].numpy()) 
        actual_class = class_names[labels[i]] 
        plt.title(f"Actual: {actual_class}, \n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        plt.axis("off")

In [ ]:
model_ver = 2
model.save(f"../models/model-ver-{model_ver}")

In [ ]:
# Get true labels and predictions
y_true = []
y_pred = []

for images, labels in test_ds:
    predictions = model.predict(images)
    y_true.extend(labels.numpy())  # True labels
    y_pred.extend(np.argmax(predictions, axis=1))  # Predicted labels

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Create confusion matrix
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)

# Plot the confusion matrix (optional)
plt.figure(figsize=(10, 8))
plt.imshow(confusion_mtx, cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.colorbar()
plt.show()